In [98]:
from sklearn import preprocessing
import numpy as np
import pandas as pd

In [99]:
DF = pd.read_csv('../data/js_viz_Data.csv')
DF.drop('index', axis=1, inplace=True)

In [89]:
nanDF = DF[pd.isnull(DF).any(axis=1)]

In [90]:
DF.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)

In [74]:
DF

,Unnamed: 0,package,github_repo,tags,search_terms,forks,stars,so_tag_counts,so_questions_counts,downloads,dl_growth_rates
0,0,plottable,palantir/plottable,[plottable.js;plottable],[plottable;plottablejs],181,2170,55,94706,36553,0.486324
1,1,taucharts,TargetProcess/tauCharts,[taucharts; taucharts.js],[taucharts;tauchartsjs],88,1361,0,4,20583,0.273709
2,2,metrics-graphics,mozilla/metrics-graphics,[metricsgraphicsjs],[metrics-graphics;metricsgraphicsjs],437,6694,13,159,11845,0.041618
3,3,dc,dc-js/dc.js,[dc.js],[dcjs;dc.js;dc-js],1551,5775,1196,1846,65422,-0.002206
4,4,smoothie,joewalnes/smoothie,[smoothie.js],[smoothie-charts;smoothie.js;smoothiejs],156,1646,9,28,4769,0.035249
5,5,keen-dataviz,keen/keen-dataviz.js,[keen-dataviz.js;keen-dataviz],[keen-dataviz],29,118,0,0,10495,-0.037949
6,6,chartist,gionkunz/chartist-js,[chartist.js],[chartist;chartistjs],1808,9744,130,186,243584,0.061352
7,7,highcharts,highcharts/highcharts,[highcharts],[highcharts;highchartsjs],2016,6526,16592,16502,1425832,0.104917
8,8,n3-charts,n3-charts/line-chart,[n3-charts],[n3-charts;n3-line-chart],200,1217,11,46,13330,-0.065470
9,9,leaflet,Leaflet/Leaflet,[leaflet],[leaflet],3248,19106,5770,5770,1007700,0.091290


In [100]:
DF_pre_scaled = DF[['forks', 'stars','so_tag_counts','so_questions_counts','downloads','dl_growth_rates']]

In [101]:
DF_scaled = pd.DataFrame(preprocessing.scale(DF_pre_scaled), columns=DF_pre_scaled.columns)

In [102]:
DF_scaled['overall_score'] = DF_scaled.sum(axis=1)
DF_scaled['package'] = DF['package']

In [94]:
DF_scaled = DF_scaled.append(nanDF[['package', 'forks', 'stars','so_tag_counts','so_questions_counts','downloads','dl_growth_rates']])

In [110]:
DF_scaled.sort_values(['overall_score'], axis=0, ascending=False, inplace=True, na_position='last')

In [111]:
DF_scaled.head()

,forks,stars,so_tag_counts,so_questions_counts,downloads,dl_growth_rates,overall_score,package
22,5.756790,5.692063,5.620210,1.597244,5.883563,-0.349363,24.200507,d3
13,2.409675,2.430622,0.384589,-0.002085,2.471051,-0.275655,7.418198,chart.js
0,-0.344741,-0.284860,-0.306866,4.489134,-0.274621,2.712016,5.990063,plottable
7,0.302258,0.117338,3.346360,0.525077,1.441155,0.093801,5.825989,highcharts
34,-0.160337,0.129803,0.114635,4.497802,-0.086568,-0.387901,4.107433,plotly.js


In [115]:
final_DF = pd.DataFrame(columns=['Package', 'Rank', 'Overall', 'Github', 'Stack Overflow', 'Downloads'])

In [127]:
final_DF['Package'] = DF_scaled['package']
final_DF['Github'] = DF_scaled[['forks', 'stars']].mean(axis=1)
final_DF['Stack Overflow'] = DF_scaled[['so_tag_counts', 'so_questions_counts']].mean(axis=1)
final_DF['Downloads'] = DF_scaled[['downloads','dl_growth_rates']].mean(axis=1)
final_DF['Overall'] = final_DF[['Github','Stack Overflow','Downloads']].sum(axis=1)
final_DF['Rank'] = final_DF['Overall'].rank(ascending=0).astype(int)

In [131]:
final_DF.style.set_table_styles([
    {'selector': '.row_heading, .blank', 'props': [('display', 'none;')]}
])

In [133]:
final_DF.to_csv('../output/js_viz_final_Rankings.csv', index = False)